# Exercise 04

In [1]:
import pandas as pd
import sqlite3

## 1. Create a connection to the database using the library sqlite3

In [2]:
connection = sqlite3.connect('../data/checking-logs.sqlite')
cursor = connection.cursor()

## 2. Using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows 

- Time should have the values: after and before
- avg_diff contains the average delta among all the users for the time period before each of them made their first visit to the page and afterward
- Only take into account the users that have observations before and after
- We still are not using the lab **’project1’**

In [ ]:
'''TEST GROUP'''

sql_query = '''
SELECT 
    time,
    AVG(diff) AS avg_diff
FROM (
    SELECT 
        uid,
        CAST(
            (strftime('%s', datetime(dl.deadlines, 'unixepoch')) - strftime('%s', t.first_commit_ts)) / 3600 
            AS INTEGER
        ) AS diff,
        CASE 
            WHEN first_commit_ts < first_view_ts THEN 'before' 
            ELSE 'after' 
        END AS time
    FROM test AS t
    JOIN deadlines AS dl 
        ON t.labname = dl.labs
    WHERE 
        t.labname != 'project1'
)
WHERE uid IN (
    SELECT uid
    FROM (
        SELECT 
            uid,
            CASE 
                WHEN first_commit_ts < first_view_ts THEN 'before' 
                ELSE 'after' 
            END AS time
        FROM test AS t
        JOIN deadlines AS dl 
            ON t.labname = dl.labs
        WHERE 
            t.labname != 'project1'
    )
    GROUP BY uid
    HAVING 
        COUNT(DISTINCT time) = 2
)
GROUP BY 
    time;
'''

df = pd.read_sql(sql_query, connection)
df

,time,avg_diff
0,after,104.6000
1,before,60.5625


In [61]:
'''CONTROL GROUP'''

sql_query = '''
SELECT 
    time,
    AVG(diff) AS avg_diff
FROM (
    SELECT 
        uid,
        CAST(
            (strftime('%s', datetime(dl.deadlines, 'unixepoch')) - strftime('%s', c.first_commit_ts)) / 3600 
            AS INTEGER
        ) AS diff,
        CASE 
            WHEN first_commit_ts < first_view_ts THEN 'before' 
            ELSE 'after' 
        END AS time
    FROM control AS c
    JOIN deadlines AS dl 
        ON c.labname = dl.labs
    WHERE 
        c.labname != 'project1'
)
WHERE uid IN (
    SELECT uid
    FROM (
        SELECT 
            uid,
            CASE 
                WHEN first_commit_ts < first_view_ts THEN 'before' 
                ELSE 'after' 
            END AS time
        FROM control AS c
        JOIN deadlines AS dl 
            ON c.labname = dl.labs
        WHERE 
            c.labname != 'project1'
    )
    GROUP BY uid
    HAVING 
        COUNT(DISTINCT time) = 2
)
GROUP BY 
    time;
'''

df = pd.read_sql(sql_query, connection)
df

,time,avg_diff
0,after,117.636364
1,before,99.464286


## 3. Close the connection

In [ ]:
connection.close()

## 4. Have the answer: did the hypothesis turn out to be true and the page does affect the students’ behavior?

The news feed probably influenced this behavior.

The test group improved 2.4 times more (+44.1 vs +18.2 hours)